<a href="https://colab.research.google.com/github/697kiran/Aerial-Object-Detection-Capstone/blob/main/YOLO_Object_Detection_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install ultralytics
from ultralytics import YOLO
import os
import yaml
from google.colab import drive
import shutil

drive.mount('/content/drive', force_remount=True)

os.environ['ULTRALYTICS_CACHE_DIR'] = '/content/ultralytics_cache'
os.makedirs(os.environ['ULTRALYTICS_CACHE_DIR'], exist_ok=True)


dataset_root_gdrive = '/content/drive/MyDrive/object_detection_Dataset'

local_dataset_path = '/content/object_detection_Dataset_local'

if os.path.exists(dataset_root_gdrive):
    print(f"Copying dataset from {dataset_root_gdrive} to {local_dataset_path}...")
    if os.path.exists(local_dataset_path):
        shutil.rmtree(local_dataset_path)
    shutil.copytree(dataset_root_gdrive, local_dataset_path)
    print(" Dataset copied locally.")
else:
    print(f" Error: Dataset root not found at {dataset_root_gdrive}. Check your path.")

yaml_path_original = os.path.join(local_dataset_path, 'data.yaml')

if os.path.exists(yaml_path_original):
    with open(yaml_path_original, 'r') as f:
        data_config = yaml.safe_load(f)

    data_config['path'] = local_dataset_path

    data_config['train'] = 'train/images'
    data_config['val'] = 'valid/images'
    data_config['test'] = 'test/images'

    new_yaml_path = os.path.join(local_dataset_path, 'data_fixed.yaml')
    with open(new_yaml_path, 'w') as f:
        yaml.dump(data_config, f)

    print(f" data.yaml configured for local Colab training. Using: {new_yaml_path}")
else:
    print(f" Error: data.yaml not found at {yaml_path_original} within the local copy. Check your dataset structure.")


model = YOLO('yolov8n.pt')

print("Starting Training...")
results = model.train(
    data=new_yaml_path,
    epochs=25,
    imgsz=640,
    batch=16,
    name='yolo_aerial_model',
    cache=False
)

metrics = model.val()
print(f"mAP50: {metrics.box.map50}")

import shutil
shutil.copy('runs/detect/yolo_aerial_model/weights/best.pt', '/content/drive/MyDrive/best_yolo.pt')
print("✅ Saved 'best_yolo.pt' to your Drive!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
Copying dataset from /content/drive/MyDrive/object_detection_Dataset to /content/object_detection_Dataset_local...
✅ Dataset copied locally.
✅ data.yaml configured for local Colab training. Using: /content/object_detection_Dataset_local/data_fixed.yaml
Starting Training...
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_past